In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)

index = -10 * np.arange(len(X))
labeled_set = LabeledSet(y, y_subspace, index)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(labeled_set.labels, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.002635608323502096, tsm: 0.0
iter: 11, fscore: 0.022690437601296597, tsm: 0.0
iter: 21, fscore: 0.10214007782101167, tsm: 0.0
iter: 31, fscore: 0.19753086419753085, tsm: 0.0
iter: 41, fscore: 0.6910299003322259, tsm: 0.005254553946753854
iter: 51, fscore: 0.5949008498583569, tsm: 0.020526960784313725
iter: 61, fscore: 0.9216589861751152, tsm: 0.25418060200668896
iter: 71, fscore: 0.9393939393939394, tsm: 0.411214953271028
iter: 81, fscore: 0.9375, tsm: 0.6486486486486487
iter: 91, fscore: 0.966824644549763, tsm: 0.8448275862068966
iter: 101, fscore: 1.0, tsm: 0.9722222222222222


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.003676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.037432,"[-829990, -282670]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.001057,0.036374,0.002636,0.000000,0.434257
2,exploration,0.037877,[-719970],[0.0],"[[0.0, 0.0]]",0.001302,0.036575,NaN,NaN,NaN
3,exploration,0.036785,[-170910],[0.0],"[[0.0, 0.0]]",0.000693,0.036092,NaN,NaN,NaN
4,exploration,0.037444,[-516460],[0.0],"[[0.0, 0.0]]",0.000682,0.036761,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
97,exploration,0.012491,[-923380],[1.0],"[[1.0, 1.0]]",0.011058,0.001433,NaN,NaN,NaN
98,exploration,0.304592,[-990530],[1.0],"[[1.0, 1.0]]",0.010714,0.293878,NaN,NaN,NaN
99,exploration,0.011776,[-580890],[1.0],"[[1.0, 1.0]]",0.010367,0.001409,NaN,NaN,NaN
100,exploration,0.158172,[-885160],[0.0],"[[1.0, 0.0]]",0.005540,0.152632,NaN,NaN,NaN


# FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(labeled_set.labels, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 1.0),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = FactorizedDualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), partition=PARTITION, mode='positive', sample_unknown_proba=0.5)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.003926774995792741, tsm: 0.0
iter: 11, fscore: 0.5213270142180095, tsm: 0.05084745762711865
iter: 21, fscore: 0.8771929824561403, tsm: 0.43902439024390244
iter: 31, fscore: 1.0, tsm: 0.65625
iter: 41, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.005774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.409944,"[-224460, -604710]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.385622,0.024322,0.003927,0.000000,0.481169
2,exploration,0.325019,[-697600],[0.0],"[[0.0, 0.0]]",0.287763,0.037256,NaN,NaN,NaN
3,exploration,0.207866,[-907620],[0.0],"[[0.0, 0.0]]",0.125265,0.082601,NaN,NaN,NaN
4,exploration,0.198872,[-507020],[0.0],"[[0.0, 0.0]]",0.196634,0.002237,NaN,NaN,NaN
5,exploration,0.016224,[-864910],[0.0],"[[0.0, 0.0]]",0.014626,0.001598,NaN,NaN,NaN
6,exploration,0.045049,[-964890],[0.0],"[[0.0, 0.0]]",0.007812,0.037237,NaN,NaN,NaN
7,exploration,0.005058,[-961780],[0.0],"[[0.0, 0.0]]",0.003965,0.001093,NaN,NaN,NaN
8,exploration,0.004838,[-149510],[0.0],"[[0.0, 1.0]]",0.003830,0.001008,NaN,NaN,NaN
9,exploration,0.002178,[-454950],[0.0],"[[1.0, 0.0]]",0.001170,0.001008,NaN,NaN,NaN
